# ArcGIS Online Basemap Replacement Notebook Update

Description: This Notebook will prompt the user for a basemap they would like to replace, and a basemap to replace it with. Then the notebook will loop through all of the web maps the user has access to, it will look for the selected basemap, and replace it with the selected replacement basemap.

Created on: 7/1/24

Purpose: The purpose of this notebook is to streamline the workflow of updating the basemaps that are used in web maps. Many users have selected our local imagery as the basemap in their web maps, and as that imagery ages, it is important to update it with the updated imagery. This will allow the user to define the basemap they want to replace, a basemap to replace it with, then loop through all of the web maps the user has access to, and update the web maps with the selected replacement basemap.

Parameters:

 - ItemID of the basemap you want to replace.

 - ItemID of the basemap you want to replace it with.

 - Select the Basemap Layer Type

Authored By: Joe Guzi

Previous Production Date: 8/14/24

Production Date: 5/21/25

Note: 
- The previous version of this notebook, was written specifically to replace Esri's Raster basemaps that were being deprecated. It can be downloaded here: https://www.arcgis.com/home/item.html?id=90cc2d7419f947d3ae55b44fd3090b24
- 8/14/24 --- updated with Audit Template
- 5/21/25 --- Updated for the ArcGIS Python API version 2.4 update
    - Supply the ItemID of the basemap you want to replace
    - Supply the ItemID of the basemap you want to replace it with
    - Select the Basemap Layer Type 

## Code Inspired By:

- ArcGIS API for Python API Reference: 
    - https://developers.arcgis.com/python/api-reference/arcgis.mapping.toc.html
    - https://developers.arcgis.com/python/api-reference/arcgis.mapping.toc.html#arcgis.mapping.WebMap.basemaps
- ArcGIS API for Python Samples: https://developers.arcgis.com/python/samples/using-and-updating-GIS-content/
- Update Basemap Layers using another Web Map - https://community.esri.com/t5/arcgis-api-for-python-questions/update-basemap-layers-using-another-web-map/td-p/1269860
- Changing basemap to a gallery_basemaps item - https://community.esri.com/t5/arcgis-api-for-python-questions/changing-basemap-to-a-gallery-basemaps-item/td-p/1089735

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.map import Map
import getpass
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
from IPython.display import display
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Choose an option below to connect to ArcGIS Online or ArcGIS Enterprise

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Import Libraries, and Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables\n",
Organization = input("What's your oranization?")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password?")
# Connection
gis = GIS(Organization, User, Password, verify_cert=False)
print(gis)
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")"

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Functions (Optional)

In [ ]:
print("Begin Function Section.")
# This is where you define Functions

print("Function Section Complete!")

# Step 4: Update Basemap in Web Map

In [ ]:
print("Begin Execute Code Section.") 
Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000) # All Web Maps
#Audit = gis.content.search(query="id:InsertItemIdForTesting", item_type="Web Map", max_items = 10000) # Item Id of a Web Map
BasemapToReplace = input("What is the ItemID of the Basemap you want to replace?")
#print(BasemapToReplace)
ItemIDBasemapReplaceWith = input("What is the ItemID of the Basemap you want to replace the previous Basemap with?")
#ItemQuery = 'id:' + ItemIDBasemapReplaceWith
#print(ItemQuery)
ReplacementBasemapItem = gis.content.get(ItemIDBasemapReplaceWith)
#print(ReplacementBasemapItem)
print("What is the Layer Type of the Basemap you want to replace the previous Basemap with?")
BaseMapLayerType = input("Select from the following options: VectorTileLayer, TileLayer, ArcGISTiledMapServiceLayer, ImageryLayer, WebTileLayer, MapImageLayer >>>> ")
#print(BaseMapLayerType)
ReplacmentBasemapDict = {
        "id" : None,
        "title" : ReplacementBasemapItem["title"],
        "itemId" : ReplacementBasemapItem["id"],
        "layerType" : BaseMapLayerType
    }
for Item in Audit:
    print(Item)
    ItemData = Item.get_data(try_json=True)
    WebMapItem = Map(Item)
    #print(WebMapItem)
    
    # Defining variables for each Item
    # New BaseMap Layer list to update Web Map
    newbaseMapLayersList = []
    # List to gather update Statuses
    UpdateStatusList= []
    # Update Status
    UpdateStatus = ""
    # Defining variables for each Item Complete!
    
    BasemapLayers = WebMapItem.basemap.layers
    #print(BasemapLayers)
    # this is where I want to fix the basemap header title in the TOC
    basemaptitle = WebMapItem.basemap.title #this gets the basemap title in the TOC
    #print(basemaptitle)
    basemaptitle = WebMapItem.basemap.title = 'Basemap' #this changes the basemap title in the TOC
    #print(basemaptitle)
    for i,basemaplayer in enumerate(BasemapLayers):
        print("In Loop of Basemap Layers")
        BasemapitemId = basemaplayer.get('itemId')
        #print(BasemapitemId)
        if BasemapitemId == BasemapToReplace:
            print("Replace BaseMap")
            BasemapID = basemaplayer.get('id')
            #print(BasemapID)
            ReplacmentBasemapDict['id'] = BasemapID
            #print(ReplacmentBasemapDict['id'])
            newbaseMapLayersList.append(ReplacmentBasemapDict)
            print("Appended Existing BaseMap to List")
            UpdateStatus = "Updated"
            UpdateStatusList.append(UpdateStatus)

        else:
            print("Did Not Replace BaseMap")
            newbaseMapLayersList.append(basemaplayer)
            print("Appended Existing BaseMap to List")
            UpdateStatus = "NotUpdated"
            UpdateStatusList.append(UpdateStatus)
    print("this is where we will update the webmap")
    #print(UpdateStatusList)
    if "Updated" in UpdateStatusList:
        print("Need to Update")
        # update basemap definition
        #print(newbaseMapLayersList)
        #print(ItemData["baseMap"]["baseMapLayers"])
        ItemData["baseMap"]["baseMapLayers"] = newbaseMapLayersList
        #print(ItemData)
        # set the item_properties dictionary
        update_properties = {"text" : ItemData}
        # update the wm_item
        print("Update the Web Map Item")
        status = Item.update(item_properties=update_properties)
        print("WebMap Updated = " + str(status))
            
    else:
        print("Item Skipped... Does not need to be updated.")
    

                      
print("Execute Code Section Complete!") 